# Embedding Food

In [2]:
# IMPORTS 
import pandas as pd 
import numpy as np 
import tensorflow as tf 
import matplotlib.pyplot as plt
import time 
import json


In [3]:
PDAT = "../../_data_tmp/food/"
PMOD = "../../dmodels/foo"
ifil = "epi_r.csv"        

In [5]:
BATCH_SIZE = 1000
USER_NUM = 20052 
ITEM_NUM = 674
DIM = 15
EPOCH_MAX = 100
DEVICE = "/cpu:0"

spn = 5000 #split

# epicurious-recipes-with-rating-and-nutrition

## Functions

In [4]:
def print_form(ds): 
    print(*ds.iloc[ds.nonzero()].index) # in 1 line
    print(*ds.iloc[ds.nonzero()])       # in 1 line
    comp = len(ds.iloc[ds.nonzero()])
    print("N. of components: {}".format(comp  ))

def print_form2(ds): 
    comp = len(ds.iloc[ds.nonzero()])
    print("N. of components: {}".format(comp  ))
    els = ds.iloc[ds.nonzero()]
    
    for i in range(len(els)): 
        print("{0:20} - {1:10}".format(els.index[i], els[i]))
    
# print_form2(dst.iloc[3])


def feed_data( json ):
    print(json)
    
    

tj = '{ }'


In [6]:
dst  =  pd.read_csv( tf.gfile.Open(PDAT + ifil), sep=None, skipinitialspace=True,  engine="python")
dst = dst.fillna(0)

In [7]:
# dst = dst.drop(2)
# dst.describe()
dst.head()
dst_n = dst["title"]
dst_x = dst.iloc[:, 6:] # everything else that are not nutritionals or ratings... Ingr, descriptions, categories ... 

In [8]:
print(len(dst_n))
dst_n.head()

20052


0                Lentil, Apple, and Turkey Wrap 
1    Boudin Blanc Terrine with Red Onion Confit 
2                  Potato and Fennel Soup Hodge 
3               Mahi-Mahi in Tomato Olive Sauce 
4                      Spinach Noodle Casserole 
Name: title, dtype: object

In [9]:
print("columns = {}".format(len(dst.columns)))
dst.columns

columns = 680


Index(['title', 'rating', 'calories', 'protein', 'fat', 'sodium', '#cakeweek',
       '#wasteless', '22-minute meals', '3-ingredient recipes',
       ...
       'yellow squash', 'yogurt', 'yonkers', 'yuca', 'zucchini', 'cookbooks',
       'leftovers', 'snack', 'snack week', 'turkey'],
      dtype='object', length=680)

In [10]:
print_form2(dst.iloc[3])
#dst_x.head()
# dst_x


N. of components: 19
title                - Mahi-Mahi in Tomato Olive Sauce 
rating               -        5.0
bon appétit          -        1.0
dairy free           -        1.0
dinner               -        1.0
fish                 -        1.0
healthy              -        1.0
high fiber           -        1.0
kosher               -        1.0
low cal              -        1.0
low fat              -        1.0
olive                -        1.0
peanut free          -        1.0
pescatarian          -        1.0
sauté                -        1.0
simmer               -        1.0
soy free             -        1.0
tomato               -        1.0
tree nut free        -        1.0


In [12]:
dst_x.head()
dst_x.describe()

,#cakeweek,#wasteless,22-minute meals,3-ingredient recipes,30 days of groceries,advance prep required,alabama,alaska,alcoholic,almond,...,yellow squash,yogurt,yonkers,yuca,zucchini,cookbooks,leftovers,snack,snack week,turkey
count,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,...,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000,20052.000000
mean,0.000299,0.000050,0.000848,0.001346,0.000349,0.005436,0.000150,0.000100,0.041642,0.029124,...,0.001247,0.026332,0.000050,0.000299,0.014861,0.000150,0.000349,0.001396,0.000948,0.022741
std,0.017296,0.007062,0.029105,0.036671,0.018681,0.073530,0.012231,0.009987,0.199774,0.168159,...,0.035288,0.160123,0.007062,0.017296,0.121001,0.012231,0.018681,0.037343,0.030768,0.149080
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [26]:
def ev_c1(): 
    pass #active
    # everything that contains : 
    l_ingr = ["apple juice", "apple", "dessert" ]
    dst.loc[ dst[ l_ingr[0] ] == 1  ] 
    
def ev_c2(): # 
    pass #active
    # everhting with more than x n (ex: 10000 calories)
    l_nut = [["calories", 10000 ], ["protein", 300]]
    index = 1
    dst.loc[ dst[ l_nut[index][0] ] > l_nut[index][1]  ] 


## Network -

* Predict the rating based on fat 
* Predict the rating based on fat and ingredients

In [ ]:
# use only dst - only 1 dataset + test

# batch function: 
def get_batches(batch_size):
    n_batches = int(len( dst.loc[spn:]  ) // batch_size)
    print(n_batches*batch_size)
    # x,y = dataT["data"][:n_batches*batch_size], dataT["label"][:n_batches*batch_size]
    for ii in range(0, len( dst.loc[spn:spn+n_batches*batch_size]) , batch_size ):
        #convert to list! 
        yield dst.iloc[spn+ii: spn+ii+batch_size, 3:].as_matrix().tolist(), dst.loc[spn+ii: spn+ii+batch_size-1, 'FP_P' ].as_matrix().tolist() 




## train

# whats_cooking_yummy --  JSON 
to do: 
* make a list of ingredients
* list of possible cuisines 
* try to predict the cuisine based on ingredients 
* try to recommend similar recipes ... (id)

In [49]:
ifiljj = "train.json"
json_str=open(PDAT + ifiljj).read();  
json_data = json.loads(json_str)

In [60]:
print(len(json_data))
type(json_data)
json_data[1]
# json_data[1]["cuisine"]

39774


{'cuisine': 'southern_us',
 'id': 25693,
 'ingredients': ['plain flour',
  'ground pepper',
  'salt',
  'tomatoes',
  'ground black pepper',
  'thyme',
  'eggs',
  'green tomatoes',
  'yellow corn meal',
  'milk',
  'vegetable oil']}

In [ ]:
#make a list of ingredients 

# Food images

In [ ]:
# !tar xzvf food-101.tar.gz
# tar.extractall(path="/new/dir/location")

import tarfile
# fname = PDAT + "food-101.tar.gz"
# tar = tarfile.open(fname, "r:gz")
# tar.extractall(path=PDAT)
# tar.close()
